# 04 — Modelagem · Film Greenlight Recommender

**Objetivo** — Prever `IMDB_Rating` (regressão) a partir de variáveis **tabulares**
pré-processadas (conversões, logs, imputações e codificações) e comparar modelos
básicos de ML, selecionando o melhor por **RMSE** (com **MAE** como métrica complementar).

**Entradas**
- Dados: `data/raw/imdb.csv`
- Pré-processador tabular: `models/preprocessor_tabular.pkl`
- Metadados: `models/meta/{num_cols_base.json, cat_cols.json, keep_genres.json}`

**Saídas**
- `models/pipeline_imdb_best.pkl` (pipeline completo: preprocessor + modelo)
- `models/model_imdb_best.pkl` (apenas o estimador)
- `reports/model_selection.csv` (comparação de modelos)
- `reports/cv_scores.csv` (validação cruzada do modelo escolhido)

**Escopo**
- Definir alvo e matriz de features em linha com o pré-processamento tabular.
- Comparar modelos: **LinearRegression**, **ElasticNet (GridSearch)**, **RandomForest** e **GradientBoosting**.
- Selecionar por **RMSE (5-fold CV)**; reportar **MAE**.
- (Opcional) Variante **Tabular + Overview (TF-IDF char 3–5)** para checar ganho marginal.
- Persistir artefatos e helper para previsão a partir de dicionário (ex.: *Shawshank*).

**Decisões**
- **Métrica principal**: RMSE (penaliza mais grandes erros), **MAE** como robusta.
- **Regularização**: ElasticNet para lidar com colinearidade e seleção suave de variáveis.
- **Evitar leakage**: todo pré-processamento dentro do `Pipeline`.

Autora: *Ana Luiza Gomes Vieira* · Execução: *Set/2025*

# Setup

In [ ]:
# Bibliotecas padrão
from pathlib import Path
import json
import re

# Bibliotecas de terceiros
import joblib
import numpy as np
import pandas as pd

from sklearn.compose import ColumnTransformer
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.linear_model import ElasticNet, LinearRegression
from sklearn.metrics import make_scorer, mean_absolute_error, mean_squared_error
from sklearn.model_selection import GridSearchCV, KFold, cross_validate, train_test_split
from sklearn.pipeline import Pipeline

In [2]:
#Configurações globais
SEED = 42
PROJECT_ROOT = Path.cwd().parent
DATA_RAW     = PROJECT_ROOT / "data" / "raw" / "imdb.csv"
MODELS_DIR   = PROJECT_ROOT / "models"
REPORTS_DIR  = PROJECT_ROOT / "reports"
FEATURES_DIR = PROJECT_ROOT / "data" / "processed"

for p in (MODELS_DIR, REPORTS_DIR, FEATURES_DIR):
    p.mkdir(parents=True, exist_ok=True)

assert DATA_RAW.exists(), f"Arquivo não encontrado: {DATA_RAW}"

In [3]:
df = pd.read_csv(DATA_RAW)

In [4]:
preprocessor_tab = joblib.load(MODELS_DIR / "preprocessor_tabular.pkl")
with open(MODELS_DIR / "meta" / "num_cols_base.json", "r", encoding="utf-8") as f:
    num_cols_base = json.load(f)
with open(MODELS_DIR / "meta" / "cat_cols.json", "r", encoding="utf-8") as f:
    cat_cols = json.load(f)
with open(MODELS_DIR / "meta" / "keep_genres.json", "r", encoding="utf-8") as f:
    keep_genres = json.load(f)

## 1. Reconstrução de Features Tabulares

### 1.1 Helpers de Conversão

In [5]:
def to_minutes(runtime: str) -> float:
    m = re.search(r"(\d+)", str(runtime))
    return float(m.group(1)) if m else np.nan

def to_usd(gross: str) -> float:
    s = str(gross).replace(",", "").strip()
    return float(s) if s and s.lower() != "nan" else np.nan

def to_decade(year) -> float:
    y = pd.to_numeric(year, errors="coerce")
    return np.floor(y/10)*10 if pd.notnull(y) else np.nan


### 1.2 Conversões/Derivações

In [6]:
df["Runtime_min"]     = df["Runtime"].apply(to_minutes)
df["Gross_usd"]       = df["Gross"].apply(to_usd)
df["Released_Year"]   = pd.to_numeric(df["Released_Year"], errors="coerce")
df["Decade"]          = df["Released_Year"].apply(to_decade)
df["No_of_Votes_log"] = np.log1p(df["No_of_Votes"])
df["Gross_usd_log"]   = np.log1p(df["Gross_usd"])

### 1.3 Gêneros (multi-hot) com as Mesmas Colunas

In [7]:
g = df["Genre"].astype(str).str.get_dummies(sep=", ")
for c in keep_genres:
    if c not in g.columns:
        g[c] = 0
genre_dum = g[keep_genres].astype(int)

### 1.4 Matriz de Features Final

In [8]:
features_df = pd.concat([df[num_cols_base], genre_dum, df[cat_cols]], axis=1)

### 1.4 Definição do Alvo e Split Holdout

In [ ]:
target = "IMDB_Rating"
data = pd.concat([features_df, df[[target]]], axis=1).dropna(subset=[target])
X = data.drop(columns=[target])
y = data[target].astype(float)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, random_state=SEED
)

## 2. Modelos e Métricas

### 2.1 Métricas (RMSE/MAE)

In [10]:
def rmse_fn(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

scoring = {
    "rmse": make_scorer(rmse_fn, greater_is_better=False),
    "mae":  make_scorer(mean_absolute_error, greater_is_better=False),
}

### 2.2 Pipelines por Modelo

In [ ]:
pipe_lr = Pipeline([("prep", preprocessor_tab), ("model", LinearRegression())])

pipe_en = Pipeline([("prep", preprocessor_tab),
                    ("model", ElasticNet(random_state=SEED, max_iter=5000))])

pipe_rf = Pipeline([("prep", preprocessor_tab),
                    ("model", RandomForestRegressor(
                        n_estimators=400, random_state=SEED, n_jobs=-1
                    ))])

pipe_gb = Pipeline([("prep", preprocessor_tab),
                    ("model", GradientBoostingRegressor(random_state=SEED))])

### 2.3 Espaço de busca (ElasticNet)

In [ ]:
param_grid_en = {
    "model__alpha":    [0.01, 0.05, 0.1, 0.3, 1.0],
    "model__l1_ratio": [0.1, 0.3, 0.5, 0.7, 0.9],
}

cv = KFold(n_splits=5, shuffle=True, random_state=SEED)

## 3. Seleção de modelo (5-fold CV)

In [13]:
rows = []

### 3.1 Avaliação dos Candidatos

In [ ]:
# LinearRegression (sem grid)
cv_lr = cross_validate(pipe_lr, X, y, scoring=scoring, cv=cv, n_jobs=-1)
rows.append({
    "model": "LinearRegression",
    "RMSE(mean)": -cv_lr["test_rmse"].mean(),
    "RMSE(std)":  cv_lr["test_rmse"].std(),
    "MAE(mean)":  -cv_lr["test_mae"].mean(),
})


In [ ]:
# ElasticNet (com grid por RMSE)
gs_en = GridSearchCV(
    pipe_en, param_grid=param_grid_en,
    scoring={"rmse": scoring["rmse"], "mae": scoring["mae"]},
    refit="rmse", cv=cv, n_jobs=-1, verbose=0
)
gs_en.fit(X_train, y_train)
best_en = gs_en.best_estimator_

cv_en = cross_validate(best_en, X, y, scoring=scoring, cv=cv, n_jobs=-1)
rows.append({
    "model": f"ElasticNet{gs_en.best_params_}",
    "RMSE(mean)": -cv_en["test_rmse"].mean(),
    "RMSE(std)":  cv_en["test_rmse"].std(),
    "MAE(mean)":  -cv_en["test_mae"].mean(),
})

In [ ]:
# RandomForest
cv_rf = cross_validate(pipe_rf, X, y, scoring=scoring, cv=cv, n_jobs=-1)
rows.append({
    "model": "RandomForest(n=400)",
    "RMSE(mean)": -cv_rf["test_rmse"].mean(),
    "RMSE(std)":  cv_rf["test_rmse"].std(),
    "MAE(mean)":  -cv_rf["test_mae"].mean(),
})


In [38]:
# GradientBoosting
cv_gb = cross_validate(pipe_gb, X, y, scoring=scoring, cv=cv, n_jobs=-1)
rows.append({
    "model": "GradientBoosting",
    "RMSE(mean)": -cv_gb["test_rmse"].mean(),
    "RMSE(std)":  cv_gb["test_rmse"].std(),
    "MAE(mean)":  -cv_gb["test_mae"].mean(),
})

### 3.2 Tabela comparativa e persistência

In [39]:
sel_df = pd.DataFrame(rows).sort_values("RMSE(mean)")
sel_df.to_csv(REPORTS_DIR / "model_selection.csv", index=False)
sel_df

,model,RMSE(mean),RMSE(std),MAE(mean)
3,GradientBoosting,0.186222,0.007782,0.148407
4,GradientBoosting,0.186222,0.007782,0.148407
2,RandomForest(n=400),0.189994,0.005732,0.151681
1,"ElasticNet{'model__alpha': 0.01, 'model__l1_ra...",0.197455,0.008659,0.158237
0,LinearRegression,0.198497,0.008915,0.158069


## 4. Treino final e validação

In [18]:
best_name = sel_df.iloc[0]["model"]

if best_name.startswith("ElasticNet"):
    best_pipe = best_en
elif best_name.startswith("RandomForest"):
    best_pipe = pipe_rf
elif best_name.startswith("GradientBoosting"):
    best_pipe = pipe_gb
else:
    best_pipe = pipe_lr

best_pipe.fit(X_train, y_train)
y_hat = best_pipe.predict(X_test)

mae  = mean_absolute_error(y_test, y_hat)
rmse = rmse_fn(y_test, y_hat)
print(f"[holdout] {best_name} | MAE={mae:.4f} | RMSE={rmse:.4f}")

[holdout] GradientBoosting | MAE=0.1548 | RMSE=0.1954


### 4.2 Salvar artefatos do modelo

In [19]:
joblib.dump(best_pipe, MODELS_DIR / "pipeline_imdb_best.pkl")
joblib.dump(best_pipe.named_steps["model"], MODELS_DIR / "model_imdb_best.pkl")
print("[ok] pipeline/model salvos em models/")

[ok] pipeline/model salvos em models/


### 4.3 Validação cruzada do modelo escolhido

In [20]:
cv_best = cross_validate(best_pipe, X, y, scoring=scoring, cv=cv, n_jobs=-1)
cv_out = pd.DataFrame({
    "RMSE": -cv_best["test_rmse"],
    "MAE":  -cv_best["test_mae"],
})
cv_out.to_csv(REPORTS_DIR / "cv_scores.csv", index=False)
cv_out.describe().loc[["mean","std"]].round(4)


,RMSE,MAE
mean,0.1862,0.1484
std,0.0087,0.0059


## 5. Inferência

In [34]:
def _load_meta(models_dir: Path):
    with open(models_dir / "meta" / "num_cols_base.json", "r", encoding="utf-8") as f:
        num_cols_base = json.load(f)
    with open(models_dir / "meta" / "cat_cols.json", "r", encoding="utf-8") as f:
        cat_cols = json.load(f)
    with open(models_dir / "meta" / "keep_genres.json", "r", encoding="utf-8") as f:
        keep_genres = json.load(f)
    return num_cols_base, cat_cols, keep_genres

### 5.1 Helper row_from_dict

In [35]:
def row_from_dict(sample: dict,
                  num_cols_base=None,
                  cat_cols=None,
                  keep_genres=None,
                  models_dir: Path | None = None) -> pd.DataFrame:

    if (num_cols_base is None) or (cat_cols is None) or (keep_genres is None):
        if models_dir is None:
            models_dir = Path.cwd().parent / "models"
            if not models_dir.exists():
                models_dir = Path("models")
        num_cols_base, cat_cols, keep_genres = _load_meta(models_dir)

    row = pd.DataFrame([sample]).copy()

    def _to_minutes(runtime: str) -> float:
        m = re.search(r"(\d+)", str(runtime))
        return float(m.group(1)) if m else np.nan

    def _to_usd(gross: str) -> float:
        s = str(gross).replace(",", "").strip()
        return float(s) if s and s.lower() != "nan" else np.nan

    def _to_decade(year) -> float:
        y = pd.to_numeric(year, errors="coerce")
        return np.floor(y/10)*10 if pd.notnull(y) else np.nan

    row["Runtime_min"]     = row.get("Runtime", np.nan).apply(_to_minutes)
    row["Gross_usd"]       = row.get("Gross", np.nan).apply(_to_usd)
    row["Released_Year"]   = pd.to_numeric(row.get("Released_Year", np.nan), errors="coerce")
    row["Decade"]          = row["Released_Year"].apply(_to_decade)
    row["No_of_Votes"]     = pd.to_numeric(row.get("No_of_Votes", np.nan), errors="coerce")
    row["No_of_Votes_log"] = np.log1p(row["No_of_Votes"])
    row["Gross_usd_log"]   = np.log1p(row["Gross_usd"])

    g = row.get("Genre", "").astype(str).str.get_dummies(sep=", ")
    for c in keep_genres:
        row[c] = g[c] if c in g.columns else 0

    for c in cat_cols:
        if c not in row.columns:
            row[c] = np.nan

    X_row = pd.concat([row[num_cols_base], row[keep_genres], row[cat_cols]], axis=1)

    expected = set(num_cols_base + keep_genres + cat_cols)
    missing  = [c for c in expected if c not in X_row.columns]
    if missing:
        raise ValueError(f"Colunas ausentes na amostra: {missing}")

    return X_row

### 5.2 Exemplo: The Shawshank Redemption

In [ ]:
shaw = {
 'Series_Title': 'The Shawshank Redemption',
 'Released_Year': 1994,
 'Certificate': 'A',
 'Runtime': '142 min',
 'Genre': 'Drama',
 'Overview': 'Two imprisoned men bond over a number of years, finding solace and eventual redemption through acts of common decency.',
 'Meta_score': 80.0,
 'Director': 'Frank Darabont',
 'Star1': 'Tim Robbins',
 'Star2': 'Morgan Freeman',
 'Star3': 'Bob Gunton',
 'Star4': 'William Sadler',
 'No_of_Votes': 2343110,
 'Gross': '28,341,469'
}

In [37]:
X_row = row_from_dict(shaw, models_dir=MODELS_DIR)
pred  = float(best_pipe.predict(X_row)[0])
print("Pred(IMDB_Rating) — Shawshank:", round(pred, 3))

Pred(IMDB_Rating) — Shawshank: 8.804


## Análise e Discussão

A modelagem teve como objetivo prever a nota `IMDB_Rating` a partir de variáveis
tabulares transformadas e derivadas no pipeline reprodutível. Foram comparados
modelos lineares e de ensemble, com a seguinte hierarquia de resultados:

- **GradientBoostingRegressor** apresentou o menor **RMSE médio (0.1862)** e
também o menor **MAE (0.1484)**, demonstrando capacidade superior de capturar
não linearidades e interações entre variáveis, mesmo em um conjunto de dados
relativamente pequeno.
- **RandomForestRegressor** obteve desempenho próximo, mas com maior variância
nos folds.
- **ElasticNet** e **LinearRegression** serviram como baselines lineares; ambos
mostraram desempenho inferior, reforçando a importância de modelos mais flexíveis.

### Fatores mais relevantes
A coerência entre os resultados da regressão e os achados do EDA reforça a
validade do pipeline:
- **`No_of_Votes_log`**: variáveis de popularidade estão fortemente associadas
às notas, refletindo o peso do engajamento do público.
- **`Gross_usd_log`**: receita de bilheteria explica parcialmente a avaliação,
mas sofre influência de fatores externos (ex.: marketing, distribuição).
- **`Meta_score`**: quando presente, fornece forte sinal de crítica especializada.
- **`Decade`** e **`Certificate`**: efeitos temporais e etários ajudam a capturar
mudanças de tendência e restrições de público.
- **Gêneros multi-hot**: adicionam variação explicativa, especialmente para
dramas, aventuras e thrillers, desde que mantidos com suporte mínimo.

### Métricas
- O **RMSE ≈ 0.19** indica que o erro típico do modelo é da ordem de **±0.2 pontos**
na escala de 1 a 10, valor adequado considerando a faixa de notas encontrada no
dataset (≈ 7.6 a 9.2).
- O **MAE ≈ 0.15** reforça a robustez contra outliers e confirma que o modelo
mantém erros absolutos médios relativamente baixos.

### Predição exemplar
Para o filme *The Shawshank Redemption*, o pipeline estimou uma nota de:
`Pred(IMDB_Rating) = 8.804`


Esse valor é altamente consistente com o status do filme como um dos mais bem
avaliados da base, dado o gênero (Drama), elevado número de votos e metascore
positivo.

### Limitações
- O modelo depende de variáveis tabulares, sem utilizar ainda a coluna textual
`Overview`, que contém sinais semânticos úteis (identificados no Notebook 02).
- Há potencial de **leakage** se variáveis derivadas não forem controladas em
produção, o que foi mitigado com persistência do pré-processador e metadados.
- A amostra disponível apresenta viés: apenas filmes populares estão na base,
com notas geralmente altas (≥ 7), limitando a generalização.

### Próximos passos
- **Integração com Overview**: plugar TF-IDF (ou embeddings como BERT) no
`ColumnTransformer`, avaliando ganhos em RMSE/MAE.
- **Modelos mais avançados**: testar CatBoost/XGBoost/LightGBM para ensembles
mais robustos em tabular + texto.
- **Tuning mais profundo**: expandir grids de hiperparâmetros para ensembles e
regularização.
- **Explainability**: aplicar SHAP ou Permutation Importance para identificar
quais variáveis contribuem mais na predição das notas.

---

## Conclusão

O pipeline desenvolvido é reprodutível, robusto e capaz de prever as notas de
filmes no IMDb com erro médio em torno de **±0.2 pontos**, o que já fornece
insumos valiosos para orientar decisões de investimento em novos projetos.
Embora o uso apenas de variáveis tabulares seja suficiente para um baseline
sólido, a inclusão do `Overview` e o refinamento de ensembles podem ampliar a
capacidade preditiva, aproximando ainda mais o sistema da complexidade do
mercado cinematográfico.

